# Analyzing Remote Sensing and *in situ* Solar Observations of the 24 March 2023 Event with `sunpy` and `pyspedas`

In [ ]:
# Needed because of last minute bug fixes to these packages
!pip install -U sunpy[all] sunpy-soar

In [ ]:
import pathlib
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u

from matplotlib.colors import LogNorm, SymLogNorm
from astropy.visualization import time_support, quantity_support, ImageNormalize, AsinhStretch, PowerStretch

import sunpy
sunpy.log.setLevel('ERROR')

DATA_DIR = pathlib.Path('./')
# In the unlikely event of network issues, uncomment this to use predownloaded data
# DATA_DIR = pathlib.Path('/home/jovyan/scratch_space/sunpy-tutorial/sunpy-pyspedas/')

In this notebook, we will be using `pyspedas` and `sunpy`, as well as several [SunPy Affiliated Packages](https://sunpy.org/affiliated/), to examine the *in situ* and remote sensing observations of the origin of a solar storm that arrived at Earth on 24 March 2023. In this notebook, we will learn how to:

- Search for and download *in situ* and remote sensing data with `sunpy`
- Create, manipulate and visualize `sunpy.timeseries.TimeSeries` objects
- Create movies from sequences of `sunpy.map.Map` objects
- Search for and download *in situ* data with `pyspedas`
- Visualize data with `pytplot` and `xarray`

By the end of this notebook, you will understand how to use `pyspedas` and `sunpy` in combination to analyze solar observations both on the solar disk and out into the solar wind.

## When did the CME happen?

Using the arrival time of the event at Earth, we can get a rough estimate of when the event occured. Let's search for LASCO data during this interval to see if we can see the eruption of the CME. LASCO, or the Large Angle and Spectrometric Coronagraph Experiment, is a coronagraph instrument on the SOHO spacecraft.

To do this we'll use the `Fido` search interface we learned about on Monday during the SunPy tutorial. We'll designate the time interval of our search using the `Time` object provided by `astropy`. Recall that we learned some basics about how to manipulate `Time` objects on Monday. We'll continue to reuse this interval throughout this demonstration.

In [ ]:
import astropy.time

In [ ]:
time_event_start = astropy.time.Time('2023-03-20 13:00:00')
time_event_end = astropy.time.Time('2023-03-20 19:00:00')

In [ ]:
from sunpy.net import Fido, attrs as a

In [ ]:
q_lasco = Fido.search(a.Time(time_event_start, end=time_event_end),
                      a.Instrument.lasco,
                      a.Detector.c2)

The cadence of the LASCO data is relatively low so let's go ahead and download every LASCO C2 file in this interval.

In [ ]:
files_lasco = Fido.fetch(q_lasco, path='data/{instrument}')

We can then load every file into a `Map` and create a `MapSequence`. `MapSequence` objects are used for storing `Map`s at subsequent time steps and are useful primarily for visualizing our 2D observations as a function of time.

In [ ]:
import sunpy.map

In [ ]:
m_lasco = sunpy.map.Map(DATA_DIR / 'data/LASCO/', sequence=True)

Let's also divide out the exposure time of each frame so that this does not influence the amount of emission per frame.

In [ ]:
m_lasco = sunpy.map.Map([m/m.exposure_time for m in m_lasco], sequence=True)

Visualizing one of the `Map` objects in our stack about halfway through our interval, we can see some very faint structures off of the limb.

In [ ]:
m_lasco[10].peek()

To bring out the structure of the CME, we can compute a running difference movie from our stack of images.

In [ ]:
m_lasco_running = sunpy.map.Map([m - prev_m.quantity for m, prev_m in zip(m_lasco[1:], m_lasco[:-1])],
                                sequence=True)

In [ ]:
ani = m_lasco_running.plot(norm=ImageNormalize(vmin=-5,vmax=5), cmap='Greys_r')

In [ ]:
HTML(ani.to_jshtml())

The SOHO spacecraft also includes an *in situ* instrument suite, including the Charge, Element, and Isotope Analysis System (CELIAS) as well as the Energetic and Relativistic Nuclei and Electron (ERNE) sensor unit. Let's use `pyspedas` to take a look at the time series and spectrogram data from these instruments and see if we can see a signature of the CME here as well.

In [ ]:
import pyspedas

Note that we've lengthened our search interval to account for the CME propagation time from the Sun to the Earth.

In [ ]:
pyspedas.soho.celias(trange=[time_event_start.iso, (time_event_end+4*u.d).iso],)

In [ ]:
pyspedas.soho.erne(trange=[time_event_start.iso, (time_event_end+4*u.d).iso])

Note that CELIAS saw an uptick in the proton density and velocity around 2023-03-23 12:00.

In [ ]:
from pytplot import tplot, options

In [ ]:
options('PH', 'spec', True)
options('PH', 'ylog', True)
options('PH', 'zlog', True)

In [ ]:
tplot(['V_p','N_p','Vth_p', 'PH'],ysize=12)

## When did the corresponding flare *actually* start?

We've now confirmed that the eruption occurred within the expected time interval. Our next question is when did this event kick off on the Sun?

To answer this, we can first search for data from the GOES XRS instrument. The GOES XRS instrument gives us the disk-integrated *soft x-ray* time series in two different energy channels. This provides an indication of when the flare started.

We can again use `Fido` and our selected time interval to search for and download GOES XRS data.

In [ ]:
q_goes = Fido.search(a.Time(time_event_start, time_event_end),
                     a.Instrument.xrs,
                     a.goes.SatelliteNumber(18),
                     a.Resolution('flx1s'))

In [ ]:
file_goes = Fido.fetch(q_goes, path='data/{instrument}/{file}')

Now, that we've downloaded the necessary GOES XRS files, we can load them into a `TimeSeries` object and visualize the resulting lightcurve. Recall that we learned the basics of creating and visualizing a `TimeSeries` object on Monday.

In [ ]:
import sunpy.timeseries

In [ ]:
ts_goes = sunpy.timeseries.TimeSeries(DATA_DIR / 'data/XRS/', concatenate=True)

In [ ]:
ts_goes.peek()

Because of the way GOES XRS data is packaged, our time series covers the whole day which clearly includes many other events. Let's use the `truncate` method to zoom in on our just the interval we are interested in.

In [ ]:
from sunpy.time import TimeRange

In [ ]:
ts_goes_flare = ts_goes.truncate(TimeRange(time_event_start, b=time_event_end))

Note that our resulting light curve for just the event of interest has a shape very characteristic of a medium-sized flare.

In [ ]:
plt.figure(figsize=(12,4))
with quantity_support():
    with time_support(simplify=True):
        plt.plot(ts_goes_flare.time, ts_goes_flare.quantity('xrsb'), color='red')
plt.show()

Let's define the peak time of our event using the maximum of the emission in this lower energy channel of GOES XRS. We'll refer to this as the peak time of the flare.

In [ ]:
time_peak = ts_goes_flare.time[ts_goes_flare.quantity('xrsb').argmax()]

In [ ]:
time_peak

## Where did this event come from?

Now that we know when the flare started, we need to know from where on the solar disk it came. Our coronagraph images obscure the solar disk and the aforementioned XRS time series are integrated over the whole disk. Fortunately, we have a suite of EUV imaging instruments pointed at the Sun that can help us answer this question.

Let's first find out where several of these instruments were. `solarmach`, a SunPy-affiliated package, makes this easy to do.

In [ ]:
import solarmach

In [ ]:
spacecraft = [
    'SDO',
    'PSP',
    'Solar Orbiter',
    'SOHO'
]

In [ ]:
smach = solarmach.SolarMACH(time_peak, spacecraft, vsw_list=[400]*len(spacecraft))

In [ ]:
smach.plot(figsize=(6.5,6.5), plot_sun_body_line=True)

We can see that *Solar Orbiter* and SDO are in a favorable configuration to view the Sun from different vantage points, providing us a multi-perspective view of the solar disk around the time when the flare went off. Notice that *Solar Orbiter* is slightly off the Sun-Earth line, meaning it provides an additional constraint from different viewing angle on the flare and eruption.

Let's first look at the HXR and SXR flux as measured by the STIX instrument on *Solar Orbiter*. STIX data is not currently provided by any of the data providers that `Fido` searches by default. However, we can use the `stixpy` package to tell `Fido` to also search the STIX data center when looking for data.

In [ ]:
from stixpy.net.client import STIXClient

In [ ]:
q_stix = Fido.search(a.Time(time_event_start, time_event_end),
                     a.Instrument.stix,
                     a.stix.DataProduct.ql_lightcurve)

In [ ]:
q_stix

In [ ]:
file_stix = Fido.fetch(q_stix, path='data/{instrument}/{file}')

After downloading our STIX data, we can then visualize these lightcurves in the exact same manner as we did with our XRS data. Again, we note that `sunpy` does not support STIX data by default, but by using `stixpy`, we can tell `TimeSeries` how to load this data and parse the accompanying metadata. 

In [ ]:
from stixpy.timeseries import quicklook

In [ ]:
ts_stix = sunpy.timeseries.TimeSeries(DATA_DIR / 'data/STIX', concatenate=True)

In [ ]:
ts_stix.peek()

As before, let's truncate our lightcurve to the interval of interest.

In [ ]:
ts_stix_flare = ts_stix.truncate(TimeRange(time_event_start, b=time_event_end))

We can overplot our STIX data on top of our GOES light

In [ ]:
fig = plt.figure(figsize=(12,4))
ax_stix = fig.add_subplot()
ax_stix.set_ylabel(f'STIX Intensity [{ts_stix_flare.units["4-10 keV"]:latex_inline}]')
ax_goes = ax_stix.twinx()
ax_goes.set_ylabel(f'STIX Intensity [{ts_goes_flare.units["xrsb"]:latex_inline}]')
with time_support(simplify=True):
    l1, = ax_stix.plot(ts_stix_flare.time, ts_stix_flare.quantity('4-10 keV'), label='STIX, 4-10 keV')
    l2, = ax_stix.plot(ts_stix_flare.time, ts_stix_flare.quantity('10-15 keV'), color='C1', label='STIX, 10-15 keV')
    l3, = ax_goes.plot(ts_goes_flare.time, ts_goes_flare.quantity('xrsb'),color='red', label='XRS, 1-8 Å')
ax_stix.set_yscale('log')
ax_goes.set_yscale('log')
ax_stix.legend([l1,l2,l3], [l.get_label() for l in [l1,l2,l3]], loc=2, frameon=False)
plt.show()

Note that these curves do *appear* to be correlated, but there is a clear offset between the two of them. **Why is that?**

Recall from our spacecraft position plot above that *Solar Orbiter* is much closer to the Sun than the Earth (GOES is in a goestationary orbit). As such emission from the Sun arrives at *Solar Orbiter* sooner than it does at GOES XRS. We can easily approximate this light travel time correction using the two spacecraft positions.

In [ ]:
from sunpy.coordinates import get_horizons_coord, get_earth

In [ ]:
earth_loc = get_earth(time_event_start)
solo_loc = get_horizons_coord('SolO', time=time_event_start)

In [ ]:
import astropy.constants as const

In [ ]:
light_travel_time = ((earth_loc.radius - solo_loc.radius) / const.c).to('s')

In [ ]:
fig = plt.figure(figsize=(12,4))
ax_stix = fig.add_subplot()
ax_stix.set_ylabel(f'STIX Intensity [{ts_stix_flare.units["4-10 keV"]:latex_inline}]')
ax_goes = ax_stix.twinx()
ax_goes.set_ylabel(f'STIX Intensity [{ts_goes_flare.units["xrsb"]:latex_inline}]')
with time_support(simplify=True):
    l1, = ax_stix.plot(ts_stix_flare.time, ts_stix_flare.quantity('4-10 keV'), label='STIX, 4-10 keV')
    l2, = ax_stix.plot(ts_stix_flare.time, ts_stix_flare.quantity('10-15 keV'), color='C1', label='STIX, 10-15 keV')
    l3, = ax_goes.plot(ts_goes_flare.time-light_travel_time, ts_goes_flare.quantity('xrsb'), color='red', label='XRS, 1-8 Å')
ax_stix.set_yscale('log')
ax_goes.set_yscale('log')
ax_stix.legend([l1,l2,l3], [l.get_label() for l in [l1,l2,l3]], loc=2, frameon=False)
plt.show()

Notice that the GOES XRS and STIX 4-10 keV light curve are highly correlated, strongly suggesting to us that both GOES and *Solar Orbiter* **are observing the same event.**

### EUV Images

Now, let's use EUV images from *Solar Orbiter* and SDO/AIA to understand the source location of this flare. We can again use `Fido` to query data from both instruments simultaneously during this time window.

In [ ]:
time_window = a.Time(time_event_start, end=time_event_end, near=time_peak)
euv_aia = a.Instrument.aia & a.Wavelength(304*u.AA)

Similar to the STIX data, `Fido` does not search the Solar Orbiter Archive (SOAR) by default. However, we can use the `sunpy_soar` package to *extend* `Fido` to also include this data source.

In [ ]:
import sunpy_soar

In [ ]:
euv_solo = a.Level(2) & (a.soar.Product('eui-fsi174-image') | a.soar.Product('eui-fsi304-image'))

In [ ]:
q_euv = Fido.search(time_window, euv_aia | euv_solo)

In [ ]:
q_euv

In [ ]:
files_euv = Fido.fetch(q_euv[:1], q_euv[1][::2], q_euv[2][::2], path='data/{instrument}/{file}')

Let's create a Map from our AIA image that was recorded near the peak of the flare.

In [ ]:
m_aia = sunpy.map.Map(DATA_DIR / 'data/AIA/aia_lev1_304a_2023_03_20t15_29_53_13z_image_lev1.fits')

We'll also correct for the degradation in the 304 channel as this can be quite severe at this point in the instrument lifetime. We can do this using the `aiapy` package, a SunPy-affiliated package for performing analysis specific to the AIA instrument.

In [ ]:
from aiapy.calibrate import correct_degradation

In [ ]:
m_aia = correct_degradation(m_aia)

In [ ]:
m_aia.peek(clip_interval=(1,99.9)*u.percent)

Let's zoom in on the part of the image where we think the flare is going off. We've manually identified this region by noting the flare "ribbons" present in our image. These features are bright in the 304 channel due to highly energetic particles impacting the cool, dense chromosphere and emitting in He II, effectively tracing out the footpoints of the structures of the flaring arcade.

We can crop to this region by specifying our bounding box in terms of the physical or *world* coordinates in the coordinate frame of this particular image.

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
blc = SkyCoord(Tx=-550*u.arcsec, Ty=300*u.arcsec, frame=m_aia.coordinate_frame)
width = 500*u.arcsec
height = 400*u.arcsec
m_aia_zoom = m_aia.submap(blc, width=width, height=height)

In [ ]:
plt.figure(figsize=(11,5))
plt.subplot(121,projection=m_aia)
m_aia.plot()
m_aia.draw_quadrangle(blc, width=width, height=height)
plt.subplot(122,projection=m_aia_zoom)
m_aia_zoom.plot()

Now, let's load all of the images from the 174 and 304 channel of EUI, the extreme ultraviolet imager on *Solar Orbter*, into two `MapSequence` objects.

In [ ]:
m_eui_174 = sunpy.map.Map(DATA_DIR / 'data/EUI/solo_L2_eui-fsi174-image_*.fits')
m_eui_304 = sunpy.map.Map(DATA_DIR / 'data/EUI/solo_L2_eui-fsi304-image_*.fits')

By using a "glob" pattern, we can load many FITS file into a list of maps. Let's take a look at just one of these.

In [ ]:
m_eui_304[18].peek()

Note that the same two-ribbon structure is visible on the disk as expected given the relative perspectives of the two spacecraft.

The field of view of the EUI Full Sun Imager (FSI) on *Solar Orbiter* is very wide. Let's crop our list of maps to the same field of view as our AIA images. Fortunately, we can reuse the bounding box for the flaring region we identifed above by using the coordinates of the bottom left and top right corners of our AIA map.

In [ ]:
roi_blc = m_aia_zoom.bottom_left_coord
roi_trc = m_aia_zoom.top_right_coord

Additionally, because the length of our observing interval is 6 hours and the desired field of view is relatively small, we need to account for the rotation of the Sun relative to the position of our spacecraft. Otherwise, the feature of interest would rotate out of view. Fortunately, `sunpy` makes this relatively easy. through the [`propagate_with_solar_surface`](https://docs.sunpy.org/en/stable/generated/api/sunpy.coordinates.propagate_with_solar_surface.html) *context manager*. This allows us to account for the effect of differential rotation of the Sun when transforming between coordinate frames defined at different times.

In [ ]:
from sunpy.coordinates import propagate_with_solar_surface

In [ ]:
with propagate_with_solar_surface():
    m_eui_304_zoom = []
    for m in m_eui_304:
        blc = roi_blc.transform_to(m.coordinate_frame)
        trc = roi_trc.transform_to(m.coordinate_frame)
        m_eui_304_zoom.append(m.submap(blc, top_right=trc))

In [ ]:
m_eui_304_zoom = sunpy.map.Map(m_eui_304_zoom, sequence=True)

Let's apply this same procedure to the 174 maps as well. This time, we'll reduce the amount of code to do this by using a list comprehension and combining a few steps. The result will be the same as above though.

In [ ]:
with propagate_with_solar_surface():
    m_eui_174_zoom = sunpy.map.Map(
        [m.submap(roi_blc.transform_to(m.coordinate_frame), top_right=roi_trc.transform_to(m.coordinate_frame)) for m in m_eui_174],
        sequence=True,
    )

Finally, let's make an animation of the flaring region as observed by EUI in the 304 and 174 channels.

In [ ]:
ani_eui_304 = m_eui_304_zoom.plot(norm=ImageNormalize(vmin=0, vmax=5e3, stretch=AsinhStretch()))
HTML(ani_eui_304.to_jshtml())

Notice that in the case of the 174 channel, the flare ribbons are much less prominent, while the post-flare arcade is much more visible. This is because this channel is much more sensitive to plasma around a 1 MK that fills the loops in the arcade during the later stage of the flare.

In [ ]:
ani_eui_174 = m_eui_174_zoom.plot(norm=ImageNormalize(vmin=0,vmax=8e3,stretch=AsinhStretch()))
HTML(ani_eui_174.to_jshtml())

### Cross-correlating *Solar Orbiter* Data

But does the emission within our selected region correspond to the event we are interested in? To answer this, we can examine the *correlation* between the emission in 174 and 304 channels of EUI and the HXR emission from STIX that we've already confirmed corresponds to our event.

To do this, we'll use our stack of images in time that correspond to the flaring region to create a `TimeSeries` of all of our *Solar Orbiter* observations. We'll first average over the spatial extent of each 304 and 174 image and subtract an estimated background.

In [ ]:
time_eui_304 = astropy.time.Time([m.date for m in m_eui_304_zoom])
ts_eui_304 = u.Quantity([m.data.mean() for m in m_eui_304_zoom], m_eui_304_zoom[0].unit)
ts_eui_304 -= ts_eui_304.min()
time_eui_174 = astropy.time.Time([m.date for m in m_eui_174_zoom])
ts_eui_174 = u.Quantity([m.data.mean() for m in m_eui_174_zoom], m_eui_174_zoom[0].unit)
ts_eui_174 -= ts_eui_174.min()

We'll also background subtract our STIX lightcurve and normalize by the exposure time.

In [ ]:
ts_stix_410 = ts_stix_flare.quantity('4-10 keV') / (ts_stix_flare.meta.metas[0]['xposure'] * u.s)
ts_stix_410 -= ts_stix_410[0]

Next, we'll interpolate each to a common time axis.

In [ ]:
t_common = np.arange(0, 6, (10*u.minute).to_value('h')) * u.h

In [ ]:
def interp_intensities(time_common, time, time_0, intensity):
    time_norm = (time - time_0).to_value('s')
    return np.interp(time_common.to_value('s'), time_norm, intensity.value) * intensity.unit

We can then use a `pandas` to easily build our `TimeSeries` object from our `dict` of columns.

In [ ]:
import pandas

In [ ]:
columns = {}
columns['time'] = (time_eui_174[0] + t_common).datetime
for time, intensity, name in [(time_eui_174, ts_eui_174, 'EUI 174 $\mathrm{\AA}$'),
                              (time_eui_304, ts_eui_304, 'EUI 304 $\mathrm{\AA}$'),
                              (ts_stix_flare.time, ts_stix_410, 'STIX 4-10 keV')]:
    columns[name] = interp_intensities(t_common, time, time_eui_174[0], intensity)
ts_solo = sunpy.timeseries.TimeSeries(pandas.DataFrame.from_dict(columns).set_index('time'))

We can then visualize this dataset in the same we did with our XRS and STIX data.

In [ ]:
ts_solo.plot()

The `sunkit-image` package, another SunPy-affiliated package, includes functionality for computing cross-correlations for any data array that includes a time dimension. We can use this functionality to compute cross-correlation curves between pairs of our intensity curves from above.

In [ ]:
from sunkit_image.time_lag import get_lags, cross_correlation

We'll compute these cross-correlations as a function of the offset in time between each curve or the *time lag* between each curve.

In [ ]:
lags = get_lags(t_common)

In [ ]:
cc_stix_304 = cross_correlation(ts_solo.quantity('STIX 4-10 keV'), ts_solo.quantity('EUI 304 $\mathrm{\AA}$'), lags,)
cc_stix_174 = cross_correlation(ts_solo.quantity('STIX 4-10 keV'), ts_solo.quantity('EUI 174 $\mathrm{\AA}$'), lags,)
cc_304_174 = cross_correlation(ts_solo.quantity('EUI 304 $\mathrm{\AA}$'), ts_solo.quantity('EUI 174 $\mathrm{\AA}$'), lags,)

All three pairs are reasonably well correlated. There is a positive delay between the STIX intensity and the 304 EUI intensity as the footpoint brightenings follow the emission of x-rays from the flare. Additionally, the post-flare loops brighten later in the 174 EUI channel as the plasma cools down to 1 MK, explaining the longer delay between STIX and EUI 174 as well as EUI 304 and 174.

In [ ]:
plt.plot(lags, cc_stix_304, label='4-10 keV, 304 $\mathrm{\AA}$')
plt.plot(lags, cc_stix_174, label='4-10 keV, 174 $\mathrm{\AA}$')
plt.plot(lags, cc_304_174, label='304 $\mathrm{\AA}$, 174 $\mathrm{\AA}$')
plt.axvline(x=0, color='k', ls=':')
plt.xlim(-3,3)
plt.legend(frameon=False)

We've quantitatively shown that emission in our limited EUI field of view corresponds to the flare observed by STIX.

## What was happening in the solar wind?

Finally, let's look out into the solar wind to see both the corresponding *in situ* data as well as an alternate view of the CME that was launched as a result of this flare.

### *in situ*

In addition to search many common sources of remote sensing data, `Fido` can also search the [NASA Coordinated Data Analysis Web (CDAWeb)](https://cdaweb.gsfc.nasa.gov/) which provides a portal for most *in situ* heliospheric data. We'll be searching CDAWeb for data from both *Parker Solar Probe* as well as *Solar Orbiter*.

To make sure our search interval includes the relavant times, we'll extend our time interval to approximately the time when the CME would have arrived at Earth.

In [ ]:
q_psp_solo = Fido.search(
    a.Time(time_event_start, time_event_end+4*u.day),
    a.cdaweb.Dataset('PSP_FLD_L2_MAG_RTN_1MIN') | a.cdaweb.Dataset('SOLO_L2_MAG-RTN-NORMAL-1-MINUTE')
)

In [ ]:
q_psp_solo

In [ ]:
!mkdir -p data/in_situ

In [ ]:
files_psp_solo = Fido.fetch(q_psp_solo, path='data/in_situ', max_conn=1)

As before, we'll create our `TimeSeries` objects by passing in paths to our data.

In [ ]:
ts_psp = sunpy.timeseries.TimeSeries(DATA_DIR / 'data/in_situ/psp_fld_*.cdf', concatenate=True)

Interestingly, the PSP magnetometer data does not seem to have observed much in the way of this event. This is likely due to the position of the spacecraft relative to the direction of propagation of the event.

In [ ]:
ts_psp.peek(marker='.')

In [ ]:
ts_solo_insitu = sunpy.timeseries.TimeSeries(DATA_DIR / 'data/in_situ/solo_l2_mag-*.cdf', concatenate=True)

In [ ]:
cme_speed = 1000*u.km / u.s

In [ ]:
time_propagation_solo = (solo_loc.radius - (1*u.R_sun)) / cme_speed

In contrast, the magnetometer data on *Solar Orbiter* shows a clear disturbance in all three magnetic field components at the approximate time at which we would expect the CME to arrive at the spacecraft. This is due to *Solar Orbiter* being more directly in the path of the CME. We can see that the disturbance in the field begins approximately when we'd expect the CME to arrive.

In [ ]:
ts_solo_insitu.plot(columns=['B_RTN_0', 'B_RTN_1', 'B_RTN_2'])
plt.axvline((time_event_start+time_propagation_solo).datetime, color='k', ls='--')

Additionally, we can use `pyspedas` to look for data from the Solar Wind Analyzer (SWA) on *Solar Orbiter*. Here, we use `pyspedas` to perform both the search and the data loading and plotting (via `xarray`) as `sunpy` does not currently have any functionality for loading multi-dimensional timeseries data as is shown here.

In [ ]:
pyspedas.solo.swa(trange=[time_event_start.iso, (time_event_end+time_propagation_solo).iso], datatype='pas-eflux')

In [ ]:
plt.figure(figsize=(12,5))
pyspedas.data_quants['eflux'].plot(y='v_dim', x='time', norm=LogNorm(vmin=1e7, vmax=5e10))

### Images

Finally, let's take a look back at PSP data, but this time looking at the image data from the Wide Field Imager for Parker Solar Probe (WISPR). WISPR is unique in that it looks to the side relative to the direction PSP is pointing in order to image structures in the solar wind.

We'll again use `Fido` to search for WISPR data, slightly lengthening our time window to account for propagation to PSP.

In [ ]:
q_wispr = Fido.search(a.Time(time_event_start, end=time_event_end+10*u.h),
                      a.Instrument.wispr,
                      a.Level(3))

In [ ]:
q_wispr

There are a lot of observations in this window so we'll only take 20% of them.

In [ ]:
files_wispr = Fido.fetch(q_wispr[0][::5], path="data/{instrument}/{file}")

We'll create two sets of `MapSequence`s, one for the inner and outer imager (denoted by the numbers appended to the filenames).

In [ ]:
m_wispr_outer = sunpy.map.Map(DATA_DIR / 'data/WISPR/*_2222.fits')
m_wispr_inner = sunpy.map.Map(DATA_DIR / 'data/WISPR/*_1221.fits')

Let's visualize the two images at one time step.

In [ ]:
wispr_norm = ImageNormalize(vmin=0, vmax=0.5e-11, stretch=PowerStretch(1/2.2))

In [ ]:
plt.figure(figsize=(10,5),layout='constrained')
plt.subplot(121, projection=m_wispr_inner[0])
m_wispr_inner[0].plot(norm=wispr_norm, cmap="viridis")
plt.subplot(122, projection=m_wispr_outer[0])
m_wispr_outer[0].plot(norm=wispr_norm, cmap="viridis")

Instead what we'd like to do is visualize data from these two cameras *together*. Fortunately, `sunpy` provides some really convenient machinery for reprojecting images into different coordinate frames. The following function takes an inner and outer WISPR image and combines them by reprojecting them to a common coordinate system. There are a lot of details in how this process works. **We do not need to worry about those at this point.**

In [ ]:
def combine_wispr_maps(inner_map, outer_map):
    from sunpy.coordinates.frames import Helioprojective
    ref_coord = SkyCoord(0*u.arcsec, 0*u.arcsec, 
                         frame=Helioprojective(observer=inner_map.observer_coordinate, obstime=inner_map.date))
    
    
    outshape = (360*2, int(360*3.5))
    new_header = sunpy.map.make_fitswcs_header(
        outshape, 
        ref_coord,
        reference_pixel=u.Quantity([40*u.pixel, 500*u.pixel]), 
        scale=u.Quantity([0.1*u.deg/u.pixel, 0.1*u.deg/u.pixel]), 
        projection_code="CAR",
    )
    import astropy.wcs
    out_wcs = astropy.wcs.WCS(new_header)
    from reproject import reproject_interp
    from reproject.mosaicking import reproject_and_coadd
    with Helioprojective.assume_spherical_screen(inner_map.observer_coordinate):
        array, footprint = reproject_and_coadd((inner_map, outer_map), out_wcs, outshape,
                                               reproject_function=reproject_interp, match_background=True)

    combined_map = sunpy.map.Map((array, new_header))
    combined_map.plot_settings["norm"] = wispr_norm
    combined_map.plot_settings["cmap"] = "viridis"
    return combined_map

Let's apply this combination procedure at one time step and visualize the resulting map with the position of the solar disk overlaid.

In [ ]:
m_wispr_combined = combine_wispr_maps(m_wispr_inner[7], m_wispr_outer[7])

In [ ]:
plt.figure(figsize=(10, 5))
m_wispr_combined.plot()
m_wispr_combined.draw_limb(color='w')

Now, let's combine the sets of images at every time step to visualize the propoagation of the CME through both cameras.

In [ ]:
m_wispr_combined = sunpy.map.Map([combine_wispr_maps(m_i, m_o) for m_i,m_o in zip(m_wispr_inner,m_wispr_outer)], sequence=True)

Finally, as with the LASCO images, we can compute a running difference movie to draw out the structure.

In [ ]:
m_wispr_running = sunpy.map.Map(
    [m - prev_m.quantity for m, prev_m in zip(m_wispr_combined[1:], m_wispr_combined[:-1])],
    sequence=True
)

In [ ]:
plt.figure(figsize=(10, 5))
ani_wispr = m_wispr_running.plot(norm=SymLogNorm(1e-14,vmin=-1e-13,vmax=1e-13), cmap='Greys_r')
m_wispr_running[0].draw_limb(color='k')

In [ ]:
HTML(ani_wispr.to_jshtml())

## Conclusion

To conclude, in this notebook we showed how to use `sunpy` and its affiliated packages, combined with `pyspedas` to analyze a flare and subsequent CME as it is initiated on the solar disk and propagates into the solar wind. In the process, we learned how to:

- Search for and download *in situ* and remote sensing data with `sunpy`
- Create, manipulate and visualize `sunpy.timeseries.TimeSeries` objects
- Create movies from sequences of `sunpy.map.Map` objects
- Search for and download *in situ* data with `pyspedas`
- Visualize data with `pytplot` and `xarray`